In [ ]:
#librerie che servono, quelle che non sono installate si installano con %pip install nome_lib
# ovviamente solo la prima volta, poi non serve più
%pip install sentence-transformers 
%pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
%pip install xlsxwriter
%pip install pandas
%pip install numpy
%pip install pandas
%pip install xlsxwriter

In [ ]:
# carichiamo le librerie necessarie

import pandas as pd
import os
import numpy as np
import torch
from sentence_transformers import SentenceTransformer
import pickle

# eliminiamo i warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# carichiamo dataset, possiamo caricare direttamente quello dettagliato generato da preparatore_DB
# il percorso è relativo a questo file

df = pd.read_csv(r'filecsv', sep=';', encoding='latin-1')
# convertiamo tutti i nan in non definito (sono i gruppi non definiti)
df = df.fillna('ND')
df.head()

In [ ]:
# poiche' tutto il procedimento sara' molto pesante, cerchiamo di levare le colonne che non ci interessano
# leviamo le colonne Descr Azienda Assegnazione, Descr Stab Assegnazione
#df = df.drop(['Descr Azienda Assegnazione', 'Descr Stab Assegnazione'], axis=1)
df = df.drop(['Descr Azienda Assegnazione'], axis=1)

In [ ]:
# Abbiamo GPU con supporto CUDA?

if torch.cuda.is_available():
    device_count = torch.cuda.device_count()
    print(f"Number of GPUs available: {device_count}")
    for i in range(device_count):
        device = torch.device(f"cuda:{i}")
        print(f"Device {i}: {torch.cuda.get_device_name(device)}")
else:
    device = torch.device("cpu")
    print("CUDA is not available")

In [ ]:
# facciamo un embeddings con algoritmo BERT e modelli preaddestrati:
# https://www.sbert.net/docs/pretrained_models.html
# all-MiniLM-L6-v2 <--- veloce e valido
# all-mpnet-base-v2 <--- lento ma piu' accurato

modelST = 'all-MiniLM-L6-v2'
model = SentenceTransformer(modelST, device=device) 

# creiamo una funzione che ci permette di fare l'embeddings (batch_size dipende da quanta RAM abbiamo)
def bert_embeddings(text):
    return model.encode(text, device = device, batch_size=64, normalize_embeddings=True) #normalize_embeddings – If set to true, returned vectors will have length 1. In that case, the faster dot-product (util.dot_score) instead of cosine similarity can be used.

ANALISI INTERO DATASET (fattibile solo se GPU con supporto CUDA presente e CPU multicore)

In [ ]:
# facciamo embedding con BERT della colonna 'Activity' del dataset 'df'
# attenzione, molto lungo, anche alcune ore, necessaria la GPU
# se già fatto e salvato, saltare e andare al punto in cui lo si carica

df['BERT_Embedding'] = df['Activity'].apply(bert_embeddings)

In [ ]:
# memorizziamo l'embedding fatto salvandolo su disco, in seguito possiamo ricaricarlo da qui 
# (se salviamo/carichiamo l'emebdding come semplice file Excel/csv non sempre è corretto)

embeddings = model.encode(df['Activity'])
#Store sentences & embeddings on disc
with open('embeddings.pkl', "wb") as fOut:
    pickle.dump({'sentences': df['Activity'], 'embeddings': df['BERT_Embedding']}, fOut, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
# carichiamo l'embedding se lo abbiamo già calcolato e salvato precedentemente
# in modo da non doverlo ricalcolare

#Load sentences & embeddings from disc
with open('embeddings.pkl', "rb") as fIn:
    stored_data = pickle.load(fIn)
    stored_sentences = stored_data['sentences']
    df['BERT_Embedding'] = stored_data['embeddings']


In [ ]:
# controlliamo dataset se e' tutto a posto
df

In [ ]:
# troviamo la media degli embeddings ottimizzato per multithread
import concurrent.futures

def calculate_mean_embedding(df):
    return np.mean(df['BERT_Embedding'])

with concurrent.futures.ThreadPoolExecutor() as executor:
    future = executor.submit(calculate_mean_embedding, df)
    embedding_avg = future.result()


In [ ]:
embedding_avg

In [ ]:
# Calculate the anomaly score for each SQL string related to embedding_avg
print('Calcolo score di anomalia...')
scores = np.dot(df['BERT_Embedding'].tolist(), embedding_avg)
# convertiamo i valori ottenuti in numeri interi ad una sola cifra
scores_abs = [int(score*10) for score in scores]
# aggiungiamo la colonna con i punteggi
df['Dataset_Anomaly'] = scores_abs
# stampa quanti valori ci sono uguali a 1
print('Numero di SQL con Dataset_Anomaly 1: ', len(df[df['Dataset_Anomaly'] == 1]))
# stampa quanti valori ci sono uguali a 2
print('Numero di SQL con Dataset_Anomaly 2: ', len(df[df['Dataset_Anomaly'] == 2]))
# stampa quanti valori ci sono uguali a 3
print('Numero di SQL con Dataset_Anomaly 3: ', len(df[df['Dataset_Anomaly'] == 3]))

In [ ]:
#salviamo i dati in file excel per test
#df.to_excel('reports/test.xlsx', index=False, engine='xlsxwriter')

ANALISI PER GRUPPO

In [ ]:
group_list = df['Descr Artle Assegnazione'].unique()
#group_list = group_list[:3] # facciamo test solo sui primi 3
print('numero di gruppi da analizzare:', len(group_list))

In [ ]:
#### routine per gruppo ####
output_file = open('reports/report_gruppi.txt', 'w')

for group in group_list:
    # Crea una cartella con il nome dell'utente se non esiste già
    group_folder = os.path.join('reports', 'gruppi', str(group))
    if not os.path.exists(group_folder):
        os.makedirs(group_folder)


    print('In elaborazione: ', group)
    output_file.write('In elaborazione: ' + group + '\n')
    #lunghezza del gruppo
    print('Numero di SQL: ', len(df[df['Descr Artle Assegnazione'] == group]))
    output_file.write('Numero di SQL: ' + str(len(df[df['Descr Artle Assegnazione'] == group])) + '\n')
    # creo un dataframe per ogni gruppo
    df_group = df[df['Descr Artle Assegnazione'] == group]
    # reset index
    df_group.reset_index(drop=True, inplace=True)

    # facciamo embedding delle 'Activity' (disattivato, usiamo quello del dataset intero)
    #print('BERT embedding delle SQL...')
    #df_group['bert_embedding'] = df_group['Activity'].apply(bert_embeddings)

    # troviamo la media degli embeddings, funzione molto veloce da generalizzare
    print('Calcolo media degli embeddings...')
    def calculate_mean_embedding(df_group):
        return np.mean(df_group['BERT_Embedding'])

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(calculate_mean_embedding, df_group)
        embedding_avg = future.result()
    #embedding_avg = np.mean(df_group['BERT_Embedding']) #molto lento

    # Calculate the anomaly score for each SQL string related to embedding_avg
    print('Calcolo score di anomalia...')
    scores = np.dot(df_group['BERT_Embedding'].tolist(), embedding_avg)
    # convertiamo i valori ottenuti in numeri interi ad una sola cifra
    scores_abs = [int(score*10) for score in scores]

    # aggiungiamo la colonna con i punteggi
    df_group['Group_Anomaly'] = scores_abs
    
    # stampa quanti valori ci sono uguali a 1
    print('Numero di SQL con Group_Anomaly 1: ', len(df_group[df_group['Group_Anomaly'] == 1]))
    output_file.write('Numero di SQL con Group_Anomaly 1: ' + str(len(df_group[df_group['Group_Anomaly'] == 1])) + '\n')
    # stampa quanti valori ci sono uguali a 2
    print('Numero di SQL con Group_Anomaly 2: ', len(df_group[df_group['Group_Anomaly'] == 2]))
    output_file.write('Numero di SQL con Group_Anomaly 2: ' + str(len(df_group[df_group['Group_Anomaly'] == 2])) + '\n')
    # stampa quanti valori ci sono uguali a 3
    print('Numero di SQL con Group_Anomaly 3: ', len(df_group[df_group['Group_Anomaly'] == 3]))
    output_file.write('Numero di SQL con Group_Anomaly 3: ' + str(len(df_group[df_group['Group_Anomaly'] == 3])) + '\n')

    # salviamo il dataframe
    print('Salvataggio in corso...')
    df_group.to_excel('reports/gruppi/' + str(group) + '/BERT_Embeddings_Score_%s.xlsx' %(group), index=False, engine='xlsxwriter')
    print('Excel salvato consuccesso! \n')

output_file.close()

ANALISI SU UTENTE SINGOLO

In [ ]:
# facciamo la stessa routine di sopra ma per utente
user_list = list(df['User'].unique())
#user_list = user_list[:20] # selezionare quanti utenti analizzare ad es: i primi 20
print('numero di utenti da analizzare:', len(user_list))

In [ ]:
#### routine per utente ####

output_file = open('reports/report_utenti.txt', 'w')

for user in user_list:
    
    # Crea una cartella con il nome dell'utente se non esiste già
    user_folder = os.path.join('reports', 'utenti', str(user))
    if not os.path.exists(user_folder):
        os.makedirs(user_folder)

    print('In elaborazione: ', user)
    output_file.write('In elaborazione: ' + user + '\n')

    # lunghezza dell'utente
    print('Stringhe SQL totali: ', len(df[df['User'] == user]))
    output_file.write('Stringhe SQL totali: ' + str(len(df[df['User'] == user])) + '\n')
    # creo un dataframe per ogni utente
    df_user = df[df['User'] == user]
    # reset index
    df_user.reset_index(drop=True, inplace=True)

    # facciamo embedding delle 'Activity'
    #print('BERT embedding delle SQL...')
    #df_user['bert_embedding'] = df_user['Activity'].apply(bert_embeddings)

    # troviamo la media degli embeddings, funzione molto veloce da generalizzare
    print('Calcolo media degli embeddings...')
    def calculate_mean_embedding(df_user):
        return np.mean(df_user['BERT_Embedding'])

    with concurrent.futures.ThreadPoolExecutor() as executor:
        future = executor.submit(calculate_mean_embedding, df_user)
        embedding_avg = future.result()
    #embedding_avg =np.mean(df_user['bert_embedding'])

    # Calculate the anomaly score for each SQL string related to embedding_avg
    print('Calcolo score di anomalia...')
    scores = np.dot(df_user['BERT_Embedding'].tolist(), embedding_avg)
    # convertiamo i valori ottenuti in numeri interi ad una sola cifra
    scores_abs = [int(score*10) for score in scores]
    # aggiungiamo la colonna con i punteggi
    df_user['User_Anomaly'] = scores_abs

    print('Numero di SQL con User_Anomaly 1: ', len(df_user[df_user['User_Anomaly'] == 1]))
    output_file.write('Numero di SQL con User_Anomaly 1: ' + str(len(df_user[df_user['User_Anomaly'] == 1])) + '\n')
    # stampa quanti valori ci sono uguali a 2
    print('Numero di SQL con User_Anomaly 2: ', len(df_user[df_user['User_Anomaly'] == 2]))
    output_file.write('Numero di SQL con User_Anomaly 2: ' + str(len(df_user[df_user['User_Anomaly'] == 2])) + '\n')
    # stampa quanti valori ci sono uguali a 3
    print('Numero di SQL con User_Anomaly 3: ', len(df_user[df_user['User_Anomaly'] == 3]))
    output_file.write('Numero di SQL con User_Anomaly 3: ' + str(len(df_user[df_user['User_Anomaly'] == 3])) + '\n' + '\n')
    
    # salviamo il dataframe
    print('Salvataggio in corso...')        
    # salviamo il file in un file Excel
    excel_filename = 'BERT_Embeddings_Score_{}.xlsx'.format(user)
    excel_path = os.path.join(user_folder, excel_filename)
    df_user.to_excel(excel_path, index=False, engine='xlsxwriter')
    print('Excel salvato consuccesso! \n')

output_file.close()